### standard imports

In [97]:
import pandas as pd
import requests
import splinter
from bs4 import BeautifulSoup as bs

In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = splinter.Browser('chrome', **executable_path, headless=False)

### scrape news

In [2]:
news_url = 'https://mars.nasa.gov/news/'

In [4]:
browser.visit(news_url)

In [5]:
soup = bs(browser.html, 'html.parser')

In [6]:
news_title = soup.body.find(class_='content_title').text
news_p = soup.body.find(class_='article_teaser_body').text

scrape img of the day

In [7]:
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)

browser.click_link_by_id('full_image')

soup = bs(browser.html, 'html.parser')

img_el = soup.body.find('img', class_='fancybox-image')

featured_img_url = f"{img_url}{img_el.attrs['src']}"

### scrape weather tweets

In [23]:
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)
soup = bs(browser.html, 'html.parser')
mars_weather = soup.body.find(class_='js-tweet-text').text

### scrape facts table w/pandas

In [36]:
facts_url = 'http://space-facts.com/mars/'
browser.visit(facts_url)
facts_df = pd.read_html(facts_url)[0].rename(columns={0:"key", 1:"value"})
facts_html = facts_df.to_html()

### scrape hemisphere imgs

In [50]:
hems_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hems_url)
soup = bs(browser.html, 'html.parser')

In [81]:
hem_titles = []
for i in soup.body.findAll(class_='product-item'):
    j = i.text.split(' ')[:-1]
    
    if(len(j) > 0):
        k = ' '.join(list(filter(lambda a: a != '', j)))
        hem_titles.append(k)

In [126]:
hem_urls = []
for i in hem_titles:
    browser.click_link_by_partial_text(i)
    soup = bs(browser.html, 'html.parser')
    hem_urls.append(soup.find(class_='downloads').a.attrs['href'])
    browser.visit(hems_url)    

In [135]:
hemisphere_image_urls = []
for i in range(4):
    hemisphere_image_urls.append({'title':hem_titles[i], 'img_url':hem_urls[i]})

In [137]:
browser.quit()